In [1]:
!pip install -q ktrain

     |████████████████████████████████| 25.3MB 121kB/s 
     |████████████████████████████████| 6.8MB 42.9MB/s 
     |████████████████████████████████| 983kB 52.2MB/s 
     |████████████████████████████████| 266kB 52.7MB/s 
     |████████████████████████████████| 1.9MB 50.1MB/s 
     |████████████████████████████████| 1.2MB 58.8MB/s 
     |████████████████████████████████| 471kB 51.7MB/s 
     |████████████████████████████████| 3.2MB 49.0MB/s 
     |████████████████████████████████| 890kB 49.5MB/s 


In [2]:
import ktrain
import numpy as np
import pandas as pd
from ktrain import text
from sklearn.model_selection import train_test_split
print(ktrain.__version__)

0.26.2


In [5]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd /content/gdrive/MyDrive/Kaggle

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/MyDrive/Kaggle


In [19]:
train_data = pd.DataFrame(pd.read_csv('train.csv'))
test_data = pd.DataFrame(pd.read_csv('test_without_target.csv'))

x_train, y_train,x_test,y_test=train_test_split(train_data.iloc[:,1:3],train_data.iloc[:,4:6],test_size=0.25,random_state=123)

In [20]:
x_train

,title,text
6684,Австрія призупинила вакцинацію однією з партій...,Австрійська влада призупинила щеплення партією...
6797,"Східна Європа має значення: голови МЗС Швеції,...",2019 рік має особливе значення для взаємин Євр...
13997,Яким буде сезон грипу в умовах COVID-19. Прогн...,Зазвичай ми вважаємо сезон грипу нормальною ча...
4678,Тимошенко намагається знову повернутися до вик...,Юлія Тимошенко / фото [SOURCE]\nЮлія Тимошенко...
14940,НБУ зберіг ставку на рівні 6% річних,Правління Національного банку ухвалило рішення...
...,...,...
12252,Boss Level: Врятувати колишню та Рая і останні...,Цього тижня в українських кінотеатрах виходять...
1346,«Залучити гроші з-під матраців». У Слузі народ...,Партія Слуга народу планує в 2021 рік провести...
11646,З початку доби бойовики чотири рази порушили п...,"З початку доби, 13 березня, бойовики на Донбас..."
15725,8 тис. грн від держави. За два дні в Дію надій...,Дія отримала понад 130 тисяч заявок на отриман...


In [7]:
y_train['text'].values.shape

(4045,)

In [8]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [11]:
 def merge(df):
   return [title + ". " + text for (title,text) in zip(df['title'],df['text'])]

In [22]:
len(merge(x_train))

12132

In [23]:
MODEL_NAME = 'youscan/ukr-roberta-base'
categories = [0, 1 , 2, 3 , 4 , 5, 6]
t = text.Transformer(MODEL_NAME, maxlen=500, classes=categories)
trn = t.preprocess_train(merge(x_train), x_test['source'].values)
val = t.preprocess_test(merge(y_train), y_test['source'].values)
model = t.get_classifier(metrics=[f1_m])
learner = ktrain.get_learner(model, train_data=trn, val_data=val, batch_size=6)

/usr/local/lib/python3.7/dist-packages/ktrain/text/preprocessor.py:414: UserWarning: The class_names argument is replacing the classes argument. Please update your code.
  warnings.warn('The class_names argument is replacing the classes argument. Please update your code.')



preprocessing train...
language: uk
train sequence lengths:
	mean : 396
	95percentile : 1335
	99percentile : 2273


Is Multi-Label? False
preprocessing test...
language: uk
test sequence lengths:
	mean : 399
	95percentile : 1303
	99percentile : 2213


404 Client Error: Not Found for url: https://huggingface.co/youscan/ukr-roberta-base/resolve/main/tf_model.h5


In [24]:
learner.fit_onecycle(5e-5, 3)



begin training using onecycle policy with max lr of 5e-05...
Epoch 1/3
2022/2022 [==============================] - 1694s 830ms/step - loss: 0.7971 - f1_m: 0.6218 - val_loss: 0.1958 - val_f1_m: 0.7681
Epoch 2/3
2022/2022 [==============================] - 1683s 831ms/step - loss: 0.2304 - f1_m: 0.7562 - val_loss: 0.1576 - val_f1_m: 0.7130
Epoch 3/3
2022/2022 [==============================] - 1683s 831ms/step - loss: 0.0994 - f1_m: 0.8169 - val_loss: 0.1210 - val_f1_m: 0.8240


In [ ]:
learner.validate(class_names=t.get_classes())

              precision    recall  f1-score   support

           0       0.96      0.96      0.96       434
           1       0.97      0.97      0.97       879
           2       0.95      0.87      0.91       259
           3       0.96      0.94      0.95       533
           4       0.93      0.96      0.95       663
           5       0.95      0.97      0.96       388
           6       0.98      0.98      0.98       889

    accuracy                           0.96      4045
   macro avg       0.96      0.95      0.95      4045
weighted avg       0.96      0.96      0.96      4045



array([[416,   2,   1,   1,   3,   7,   4],
       [  1, 857,   0,   7,   3,   1,  10],
       [  0,   0, 225,   0,  34,   0,   0],
       [  6,   7,   3, 501,   6,   8,   2],
       [  3,   1,   7,   9, 638,   5,   0],
       [  5,   3,   2,   2,   1, 375,   0],
       [  4,  16,   0,   2,   0,   0, 867]])

In [ ]:
learner.view_top_losses(n=1, preproc=t)

----------
id:3839 | loss:8.63 | true:0 | pred:2)



In [25]:
predictor = ktrain.get_predictor(learner.model, preproc=t)

In [26]:
predictor.save('/predictor')

In [28]:
predicted = predictor.predict(merge(test_data))

In [29]:
ids = test_data.iloc[:,[0]]
ids = list(map(lambda x: int(x),np.array(ids)))
res = zip(ids, predicted)
res = list(map(lambda x: {"Id": x[0], "Predicted": int(x[1])}, list(res)))
res
import csv
from google.colab import files
with open("result.csv", "w", newline="") as file:
  columns = ["Id", "Predicted"]
  writer = csv.DictWriter(file, fieldnames=columns)
  writer.writeheader()
  print(len(res))
  writer.writerows(res)
  files.download("result.csv")

16183


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>